In [33]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from fanalysis.ca import CA
from matplotlib.ticker import PercentFormatter

# Préparation des données. 

In [36]:
# On charge les bases de données. 
data_sn1 = pd.read_csv('data/data-sn1.csv')
cross_data_table_sn1 = pd.read_csv('data/id_sn1.csv')
# cross_data_table_sn2 = pd.read_csv('data/id_sn2.csv')
items = pd.read_csv('data/items.csv')

# On renomme les colonnes pour éviter les accents, espaces et caractères spéciaux. 
data_sn1 = data_sn1.rename(columns={
    "N° interne": "student_id_short",
    "N° SA": "record_id_short",
    "SN1/SN2": "tag",
    " Famille  de situation" : "FS",
    "Problématique exemplaire" : "PE",
    "Urgences-SN1" : "order_1",
    "Ped Ambu": "ped_ambu",
    "SN2-Gynéco" : "order_2",
    "Titre" : "title", 
    "Spécialité" : "specialty", 
    "5 types d'étapes de raisonnement": "step_5",
    "2 types d'étapes du raisonnement": "step_2",
    "Item" : "ecn_item", 
    "Compétences" : "skill",
    "Compétences certifiées": "certified_skills_list"
})

# On renomme les problématiques exemplaires. 
data_sn1.loc[data_sn1['PE'] == "Situations cliniques inédites de soin, de prévention, de dépistage et d'éducation, occasion d'un apprentissage", 'PE'] = 'PE01'
data_sn1.loc[data_sn1['PE'] == "Prescriptions médicamenteuses nouvelles", 'PE'] = 'PE02'
data_sn1.loc[data_sn1['PE'] == "Incidents iatrogènes nouvellement rencontrés", 'PE'] = 'PE03'
data_sn1.loc[data_sn1['PE'] == "Difficultés relationnelles avec le patient ou son entourage", 'PE'] = 'PE04'
data_sn1.loc[data_sn1['PE'] == "Procédures médico-administratives", 'PE'] = 'PE05'
data_sn1.loc[data_sn1['PE'] == "Situations de soins non programmés vous ayant posé problème", 'PE'] = 'PE06'
data_sn1.loc[data_sn1['PE'] == "Pratique de gestes techniques utiles en soins primaires", 'PE'] = 'PE07'
data_sn1.loc[data_sn1['PE'] == "Evaluation par le sénior de la compétence à communiquer avec le patient selon les critères de la grille de Calgary simplifiée", 'PE'] = 'PE08'
data_sn1.loc[data_sn1['PE'] == "Exemples de l'apport d'articles de la littérature utiles pour réévaluer la décision", 'PE'] = 'PE09'
data_sn1.loc[data_sn1['PE'] == "Exemples de l'apport de la supervision de l'enseignant clinicien modifiant votre perception de la situation", 'PE'] = 'PE10'

# On renomme les compétences. 
data_sn1.loc[data_sn1['skill'] == "Relation, communication, approche centrée patient", 'skill'] = 'C1'
data_sn1.loc[data_sn1['skill'] == "Approche globale, prise en compte de la complexité", 'skill'] = 'C2'
data_sn1.loc[data_sn1['skill'] == "Éducation, prévention, dépistage, santé individuelle et communautaire", 'skill'] = 'C3'
data_sn1.loc[data_sn1['skill'] == "Premier recours, urgences", 'skill'] = 'C4'
data_sn1.loc[data_sn1['skill'] == "Continuité, suivi, coordination des soins autour du patient", 'skill'] = 'C5'
data_sn1.loc[data_sn1['skill'] == "Professionnalisme", 'skill'] = 'C6'

# Permet de remplacer les intitulés des item par les numéros. 
for index, row in items.iterrows():
    data_sn1.loc[data_sn1['ecn_item'] == row[1], 'ecn_item'] = f"Item {row[0]}"

# On supprime les valeurs 0, 1 et 2 de la variable RC qui ne correspondent pas à des RC. 
data_sn1.loc[data_sn1['RC'] == "0", 'RC'] = np.nan
data_sn1.loc[data_sn1['RC'] == "1", 'RC'] = np.nan
data_sn1.loc[data_sn1['RC'] == "2", 'RC'] = np.nan

# On supprime les valeurs 0 et ? qui ne sont pas des item ECN. 
data_sn1.loc[data_sn1['ecn_item'] == "0", 'ecn_item'] = np.nan
data_sn1.loc[data_sn1['ecn_item'] == "?", 'ecn_item'] = np.nan

# On réintroduit les identifiants complets de l'étudiant, du stage et du SA (dans le cas où des extractions complémentaires seraient nécessaires). 
data_sn1 = pd.merge(data_sn1, cross_data_table_sn1, on="record_id_short")

data_sn1 = data_sn1[['student_id', 'internship_id', 'record_id', "tag", "FS", "PE", "RC", "order_1", "order_2", "ped_ambu", "specialty", "step_5", "step_2", "ecn_item", "skill"]]

# Création des colonnes step_PEC_TTT, step_diag et step_autre. La valeur par défaut est 0.
data_sn1['step_PEC_TTT'] = 0 
data_sn1['step_diag'] = 0
data_sn1['step_autre'] = 0

# En fonction des valeurs de la colonne step_2, on modifie les valeurs des colonnes step_### correspondante. 
data_sn1.loc[data_sn1['step_2'] == 'PEC-ttt', 'step_PEC_TTT'] = 1

data_sn1.loc[data_sn1['step_2'] == 'Diagnostic', 'step_diag'] = 1 

data_sn1.loc[data_sn1['step_2'] == 'Diag+PEC', 'step_PEC_TTT'] = 1
data_sn1.loc[data_sn1['step_2'] == 'Diag+PEC', 'step_diag'] = 1

data_sn1.loc[data_sn1['step_2'] == 'Autre', 'step_autre'] = 1

data_sn1.to_csv('data.csv')
display(data_sn1)

/var/folders/p9/3j1vdwh52f7620pmfxc9l1y00000gn/T/ipykernel_11524/862333343.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data_sn1.loc[data_sn1['ecn_item'] == row[1], 'ecn_item'] = f"Item {row[0]}"
/var/folders/p9/3j1vdwh52f7620pmfxc9l1y00000gn/T/ipykernel_11524/862333343.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data_sn1.loc[data_sn1['ecn_item'] == row[1], 'ecn_item'] = f"Item {row[0]}"
/var/folders/p9/3j1vdwh52f7620pmfxc9l1y00000gn/T/ipykernel_11524/862333343.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated a

,student_id,internship_id,record_id,tag,FS,PE,RC,order_1,order_2,ped_ambu,specialty,step_5,step_2,ecn_item,skill,step_PEC_TTT,step_diag,step_autre
0,1ed8105d-c254-6f7a-9ce4-91b15023c51a,1eddd3e3-68f7-683e-afb1-37d1566b753d,1ee19e86-0b9d-6596-be29-cdb02722aed8,SN1-1,FS01,PE10,DIABÈTE DE TYPE 2,Urgences avant SN1,SN2 avant Pôle femme,non,Endocrinologie,PEC-ttt,PEC-ttt,Item 247,C5,1,0,0
1,1ed8105d-c254-6f7a-9ce4-91b15023c51a,1eddd3e3-68f7-683e-afb1-37d1566b753d,1ee19e91-e5b2-6276-a638-9d5d9e29e54c,SN1-1,FS08,PE05,PSYCHIQUE (TROUBLE),Urgences avant SN1,SN2 avant Pôle femme,non,Psychiatrie,PEC-ttt,PEC-ttt,Item 182,C2,1,0,0
2,1ed8105d-c254-6f7a-9ce4-91b15023c51a,1eddd3e3-68f7-683e-afb1-37d1566b753d,1ee1ea5d-5f54-6d2e-aff6-7b9b23d3a68d,SN1-1,FS09,PE04,NaN,Urgences avant SN1,SN2 avant Pôle femme,non,Ethique/Communication/Relation,Ethique-Communication-Relation,Autre,Item 1,C1,0,0,1
3,1ed8105d-c254-6f7a-9ce4-91b15023c51a,1eddd3e3-68f7-683e-afb1-37d1566b753d,1ee1ea60-c66d-6242-b8d0-69798fe3aa18,SN1-1,FS02,PE01,RÉACTION TUBERCULINIQUE,Urgences avant SN1,SN2 avant Pôle femme,non,Pneumologie,Diagnostic,Diagnostic,Item 159,C3,0,1,0
4,1ed8105d-c254-6f7a-9ce4-91b15023c51a,1eddd3e3-68f7-683e-afb1-37d1566b753d,1ee20792-4870-6322-bfee-19c31d73b3b6,SN1-1,FS02,PE01,AMAIGRISSEMENT,Urgences avant SN1,SN2 avant Pôle femme,non,Médecine Interne - Immuno,Diagnostic,Diagnostic,Item 251,C2,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,1ed8105d-c2a8-6e72-b93d-91b15023c51a,1eddd3c1-4967-63f6-9a68-15b8171db13d,1ee1ce8a-d256-6460-af25-51df10b19615,SN1-1,FS09,PE04,NaN,Urgences avant SN1,SN2 avant Pôle femme,non,Ethique/Communication/Relation,Ethique-Communication-Relation,Autre,Item 1,C1,0,0,1
599,1ed8105d-c2a8-6e72-b93d-91b15023c51a,1eddd3c1-4967-63f6-9a68-15b8171db13d,1ee1ceba-4883-6078-9391-ab433d0c33ed,SN1-1,FS02,PE07,PROCÉDURE ADMINISTRATIVE,Urgences avant SN1,SN2 avant Pôle femme,non,Santé publique (Administratif/Social),Administratif-Social,Autre,Item 256,C3,0,0,1
600,1ed8105d-c2a8-6e72-b93d-91b15023c51a,1eddd3c1-4967-63f6-9a68-15b8171db13d,1ee24ddd-d22d-625e-b485-0905d08c7152,SN1-1,FS02,PE03,ANOMALIE BIOLOGIQUE SANGUINE,Urgences avant SN1,SN2 avant Pôle femme,non,Neurologie,Diagnostic,Diagnostic,Item 96,C2,0,1,0
601,1ed8105d-c2a8-6e72-b93d-91b15023c51a,1eddd3c1-4967-63f6-9a68-15b8171db13d,1ee5edc6-85a9-6b8a-a74b-a1d05b96f9bc,SN1-1,FS04,PE08,NaN,Urgences avant SN1,SN2 avant Pôle femme,non,Ethique/Communication/Relation,Ethique-Communication-Relation,Autre,Item 1,C1,0,0,1


## Recodage étape (diagnostic, prise en charge, traitement)

In [11]:


display(data_sn1)


,student_id,internship_id,record_id,tag,FS,PE,RC,order_1,order_2,ped_ambu,specialty,step_5,step_2,ecn_item,skill,step_PEC_TTT,step_diag,step_autre
0,1ed8105d-c254-6f7a-9ce4-91b15023c51a,1eddd3e3-68f7-683e-afb1-37d1566b753d,1ee19e86-0b9d-6596-be29-cdb02722aed8,SN1-1,FS01,PE10,DIABÈTE DE TYPE 2,Urgences avant SN1,SN2 avant Pôle femme,non,Endocrinologie,PEC-ttt,PEC-ttt,Item 247,C5,1,0,0
1,1ed8105d-c254-6f7a-9ce4-91b15023c51a,1eddd3e3-68f7-683e-afb1-37d1566b753d,1ee19e91-e5b2-6276-a638-9d5d9e29e54c,SN1-1,FS08,PE05,PSYCHIQUE (TROUBLE),Urgences avant SN1,SN2 avant Pôle femme,non,Psychiatrie,PEC-ttt,PEC-ttt,Item 182,C2,1,0,0
2,1ed8105d-c254-6f7a-9ce4-91b15023c51a,1eddd3e3-68f7-683e-afb1-37d1566b753d,1ee1ea5d-5f54-6d2e-aff6-7b9b23d3a68d,SN1-1,FS09,PE04,NaN,Urgences avant SN1,SN2 avant Pôle femme,non,Ethique/Communication/Relation,Ethique-Communication-Relation,Autre,Item 1,C1,0,0,1
3,1ed8105d-c254-6f7a-9ce4-91b15023c51a,1eddd3e3-68f7-683e-afb1-37d1566b753d,1ee1ea60-c66d-6242-b8d0-69798fe3aa18,SN1-1,FS02,PE01,RÉACTION TUBERCULINIQUE,Urgences avant SN1,SN2 avant Pôle femme,non,Pneumologie,Diagnostic,Diagnostic,Item 159,C3,0,1,0
4,1ed8105d-c254-6f7a-9ce4-91b15023c51a,1eddd3e3-68f7-683e-afb1-37d1566b753d,1ee20792-4870-6322-bfee-19c31d73b3b6,SN1-1,FS02,PE01,AMAIGRISSEMENT,Urgences avant SN1,SN2 avant Pôle femme,non,Médecine Interne - Immuno,Diagnostic,Diagnostic,Item 251,C2,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,1ed8105d-c2a8-6e72-b93d-91b15023c51a,1eddd3c1-4967-63f6-9a68-15b8171db13d,1ee1ce8a-d256-6460-af25-51df10b19615,SN1-1,FS09,PE04,NaN,Urgences avant SN1,SN2 avant Pôle femme,non,Ethique/Communication/Relation,Ethique-Communication-Relation,Autre,Item 1,C1,0,0,1
599,1ed8105d-c2a8-6e72-b93d-91b15023c51a,1eddd3c1-4967-63f6-9a68-15b8171db13d,1ee1ceba-4883-6078-9391-ab433d0c33ed,SN1-1,FS02,PE07,PROCÉDURE ADMINISTRATIVE,Urgences avant SN1,SN2 avant Pôle femme,non,Santé publique (Administratif/Social),Administratif-Social,Autre,Item 256,C3,0,0,1
600,1ed8105d-c2a8-6e72-b93d-91b15023c51a,1eddd3c1-4967-63f6-9a68-15b8171db13d,1ee24ddd-d22d-625e-b485-0905d08c7152,SN1-1,FS02,PE03,ANOMALIE BIOLOGIQUE SANGUINE,Urgences avant SN1,SN2 avant Pôle femme,non,Neurologie,Diagnostic,Diagnostic,Item 96,C2,0,1,0
601,1ed8105d-c2a8-6e72-b93d-91b15023c51a,1eddd3c1-4967-63f6-9a68-15b8171db13d,1ee5edc6-85a9-6b8a-a74b-a1d05b96f9bc,SN1-1,FS04,PE08,NaN,Urgences avant SN1,SN2 avant Pôle femme,non,Ethique/Communication/Relation,Ethique-Communication-Relation,Autre,Item 1,C1,0,0,1
